In [1]:

import collections
import numpy as np
from keras.layers import Input, Dense, Bidirectional, LSTM
from keras.models import Model
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.layers.embeddings import Embedding
import tensorflow as tf

#Load Data

In [2]:
inputsents=[]
cnsents=[]
engsents=[]
with open('/content/cmn.txt','r') as f:
  for line in f.readlines():
    pair=line.split('\t')[:2]
    pair[0]=pair[0].rstrip()
    pair[0]=pair[0].lstrip()
    pair[1]=pair[1].rstrip()
    pair[1]=pair[1].lstrip()
    inputsents.append(pair)
    cnsents.append(pair[1])
    engsents.append(pair[0])
inputsents[:5]

[['Hi.', '嗨。'],
 ['Hi.', '你好。'],
 ['Run.', '你用跑的。'],
 ['Wait!', '等等！'],
 ['Wait!', '等一下！']]

In [3]:
print(len(cnsents))
print(len(engsents))
num_of_sents=len(cnsents)
max_len_cn=max([len(s) for s in cnsents])
max_len_eng=max([len(s) for s in engsents])
print('max length of chinese sentences', max_len_cn)
print('max length of english sentences', max_len_eng)

24360
24360
max length of chinese sentences 44
max length of english sentences 163


#Preprocessing

##Tokenize Sentences

In [4]:
from keras.preprocessing.text import Tokenizer

def tokenize(x, vocab_size=10000, encode_start_end = False):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :param encode_start_end: if True, pad the start & end of sentence as separate tokens
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    if encode_start_end:
        x = ["startofsentence " + sentence + " endofsentence" for sentence in x]
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    tokenized_x = tokenizer.texts_to_sequences(x)
    
    return tokenized_x, tokenizer

In [5]:
cn_tokenized, cn_tokenizer=tokenize(cnsents)
en_tokenized, en_tokenizer=tokenize(engsents, encode_start_end=True)

##padding

In [6]:
from keras.preprocessing.sequence import pad_sequences

def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if length is None:
        length = max([len(sentence) for sentence in x])
        
    padded_x = pad_sequences(x, maxlen = length, padding = 'post', truncating = 'post')
    
    return padded_x

In [7]:
padded_cn=pad(cn_tokenized, max_len_cn)
padded_en=pad(en_tokenized, max_len_eng)

cn_vocab_size=len(cn_tokenizer.word_index)
en_vocab_size=len(en_tokenizer.word_index)

print('Chinese vocab size ', cn_vocab_size)
print('English vocab size ', en_vocab_size)

print('Chinese padded sequence length', len(padded_cn[0]))
print('English padded sequence length', len(padded_en[0]))

print(padded_cn.shape)
print(padded_en.shape)
print((en_tokenizer.word_index))

Chinese vocab size  22429
English vocab size  6738
Chinese padded sequence length 44
English padded sequence length 163
(24360, 44)
(24360, 163)
{'startofsentence': 1, 'endofsentence': 2, 'the': 3, 'i': 4, 'to': 5, 'you': 6, 'a': 7, 'is': 8, 'tom': 9, 'he': 10, 'in': 11, 'of': 12, 'my': 13, 'this': 14, 'me': 15, 'do': 16, 'it': 17, 'that': 18, 'have': 19, 'she': 20, 'was': 21, 'for': 22, "don't": 23, 'your': 24, 'are': 25, 'on': 26, 'we': 27, 'his': 28, 'what': 29, 'at': 30, 'like': 31, "i'm": 32, 'be': 33, 'with': 34, 'not': 35, 'can': 36, 'go': 37, 'her': 38, 'know': 39, 'want': 40, 'and': 41, 'has': 42, 'will': 43, 'how': 44, 'there': 45, 'him': 46, 'very': 47, 'as': 48, "it's": 49, 'time': 50, 'all': 51, 'did': 52, 'please': 53, 'here': 54, 'they': 55, 'up': 56, 'good': 57, 'no': 58, 'think': 59, 'an': 60, 'by': 61, 'about': 62, 'had': 63, 'from': 64, 'out': 65, 'mary': 66, 'get': 67, "can't": 68, "didn't": 69, 'when': 70, 'going': 71, 'would': 72, 'come': 73, 'if': 74, 'should': 7

#Build Model

In [8]:
en_target=padded_en[:,1:,]
padded_cn[50]

array([1665,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

In [15]:
#encoder

cn_input=padded_cn
en_input=padded_en[:,:-1,]
en_target=padded_en[:,1:,]



encoder_input=Input(shape=(None,), name='encoder_input')
emb_dim=200
embedding=Embedding(cn_vocab_size+1, emb_dim, mask_zero=True)
cn_embedding=embedding(encoder_input)
encoder_lstm=LSTM(256, activation='relu', return_state=True, name='encoder_LSTM')

encoder_outputs, last_hidden, last_cell_state=encoder_lstm(cn_embedding)

#decoder
decoder_input=Input(shape=(None, ), name='decoder_input')
embedding=Embedding(en_vocab_size+1, emb_dim, mask_zero=True)
en_embedding=embedding(decoder_input)

decoder_lstm=LSTM(256, activation='relu', return_sequences=True, return_state=True, name='decoder_LSTM')
decoder_outputs, _,_=decoder_lstm(en_embedding, initial_state=[last_hidden, last_cell_state])
decoder_dense=Dense(en_vocab_size+1, activation='softmax', name='decoder_dense')
logits=decoder_dense(decoder_outputs)

model=Model([encoder_input, decoder_input], logits)

model.compile(loss = 'sparse_categorical_crossentropy',
                        optimizer = Adam(lr = 0.002),
                        metrics = ['accuracy'])

model.fit([cn_input, en_input],
                    en_target,
                    batch_size = 128,
                    epochs = 20,
                    validation_split = 0.2)


Epoch 1/20
153/153 [==============================] - 601s 4s/step - loss: 0.3034 - accuracy: 0.2212 - val_loss: 0.4271 - val_accuracy: 0.2056
Epoch 2/20
153/153 [==============================] - 587s 4s/step - loss: 0.1915 - accuracy: 0.3326 - val_loss: 0.4520 - val_accuracy: 0.2301
Epoch 3/20
153/153 [==============================] - 586s 4s/step - loss: 0.1655 - accuracy: 0.3718 - val_loss: 0.5057 - val_accuracy: 0.2406
Epoch 4/20
153/153 [==============================] - 594s 4s/step - loss: 0.1491 - accuracy: 0.3959 - val_loss: 0.9267 - val_accuracy: 0.2311
Epoch 5/20
153/153 [==============================] - 596s 4s/step - loss: 0.1351 - accuracy: 0.4206 - val_loss: 1.3803 - val_accuracy: 0.2337
Epoch 6/20
153/153 [==============================] - 588s 4s/step - loss: 0.1227 - accuracy: 0.4474 - val_loss: 1.7876 - val_accuracy: 0.2339
Epoch 7/20
153/153 [==============================] - 585s 4s/step - loss: 0.1088 - accuracy: 0.4889 - val_loss: 1.9398 - val_accuracy: 0.2288

In [16]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
decoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 200)    4486000     encoder_input[0][0]              
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 200)    1347800     decoder_input[0][0]              
____________________________________________________________________________________________

In [11]:
decoder_lstm.input_spec

[InputSpec(shape=(None, None, 200), ndim=3)]

# Inference Model

In [17]:
#encoder
last_states_encoder = [last_hidden, last_cell_state]
inference_encoder_model = Model(encoder_input, 
                                last_states_encoder)

#decoer
decoder_initial_state = [Input(shape = (256,)), Input(shape = (256,))]
all_hidden_decoder, last_hidden_decoder, last_cell_decoder = decoder_lstm(embedding(decoder_input), 
                                                                          initial_state = decoder_initial_state)

logits = decoder_dense(all_hidden_decoder)

inference_decoder_model = Model([decoder_input] + decoder_initial_state, 
                                [logits, 
                                          last_hidden_decoder, 
                                          last_cell_decoder])



In [18]:
target_id_to_word = {idx:word for word, idx in en_tokenizer.word_index.items()}

def decode_sequence(input_seq):
    """
    Gets predictions using the final model defined above
    :param input_seq: (list) encoded english sentence (list of word ids)
    returns : translated French sentence
    """
    states_value = inference_encoder_model.predict(input_seq)
    # Initialize decoder input as a length 1 sentence containing "startofsentence",
    # --> feeding the start token as the first predicted word
    prev_word = np.zeros((1, 1, 1))
    prev_word[0, 0, 0] = en_tokenizer.word_index["startofsentence"]

    stop_condition = False
    translation = []
    decoded_sentence=[]
    while not stop_condition:
        # 1. predict the next word using decoder model
        logits, last_h, last_c = inference_decoder_model.predict([prev_word] + states_value)
        
        # 2. Update prev_word with the predicted word
        predicted_id = np.argmax(logits[0, 0, :])
        #print(decoder_input)
        #print(logits)
        predicted_word = target_id_to_word[predicted_id]
        decoded_sentence.append(predicted_word)

        # 3. Enable End Condition: (1) if predicted word is "endofsentence" OR
        #                          (2) if translated sentence reached maximum sentence length
        if (predicted_word == 'endofsentence' or len(translation) > en_target.shape[1]):
            stop_condition = True

        # 4. Update prev_word with the predicted word
        prev_word[0, 0, 0] = predicted_id

        # 5. Update initial_states with the previously predicted word's encoder output
        states_value = [last_h, last_c]

    return " ".join(decoded_sentence).replace('endofsentence', '')

In [19]:
print(cnsents[240])

不要动。


In [21]:
# TODO: Print prediction(s)
for i in [3000]:
    cn_seq = padded_cn[i].reshape(1, padded_cn.shape[1])
    print(cn_seq)
    eng_translation = decode_sequence(cn_seq)
    
    print("Chinese Sentence            : ", cnsents[i])
    print("Predicted English Translation: ", eng_translation)
    print("Correct English Translation  : ", engsents[i])
    print()

[[3817    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]]
Chinese Sentence            :  她很可爱。
Predicted English Translation:  she is very pretty 
Correct English Translation  :  She is very pretty.

